### Import Packages and Set Global Variables

In [ ]:
import time
import math
import copy
import torch
import pickle
import random
import warnings
import numpy as np
import pandas as pd
import scienceplots
import torch.nn as nn
import torch.optim as opt
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from copy import deepcopy
from torch.autograd import grad
from scipy.stats import spearmanr
from sklearn.linear_model import SGDClassifier
from matplotlib.ticker import FormatStrFormatter
from folktables import ACSDataSource, ACSPublicCoverage
from sklearn.metrics import mean_absolute_error, log_loss, accuracy_score

warnings.filterwarnings("ignore")

E = math.e

### Utility Functions

#### Get and transform 4 class mnist

In [ ]:
def get_MNIST():
    train_dataset = dsets.MNIST(root='', train=True, transform=transforms.ToTensor(), download=True)
    test_dataset = dsets.MNIST(root='', train=False, transform=transforms.ToTensor(), download=True)
    
    idx = (train_dataset.targets==1) | (train_dataset.targets==7) | (train_dataset.targets==3) | (train_dataset.targets==8)
    train_dataset.targets = train_dataset.targets[idx]
    train_dataset.data = train_dataset.data[idx]
    
    new_targets = []
    
    for old_lab in train_dataset.targets:
        if old_lab == 1:
            new_targets.append(0)
        elif old_lab == 3:
            new_targets.append(1)
        elif old_lab == 7:
            new_targets.append(2)
        elif old_lab == 8:
            new_targets.append(3)
            
    train_dataset.targets = new_targets
    
    idx = (test_dataset.targets==1) | (test_dataset.targets==7) | (test_dataset.targets==3) | (test_dataset.targets==8)
    test_dataset.targets = test_dataset.targets[idx]
    test_dataset.data = test_dataset.data[idx]
    
    new_targets = []
    
    for old_lab in test_dataset.targets:
        if old_lab == 1:
            new_targets.append(0)
        elif old_lab == 3:
            new_targets.append(1)
        elif old_lab == 7:
            new_targets.append(2)
        elif old_lab == 8:
            new_targets.append(3)
            
    test_dataset.targets = new_targets
    print(len(train_dataset))
    return train_dataset, test_dataset

#### Plot results

In [ ]:
def visualize_result(e_k_actual, e_k_estimated, ep, k_):
    plt.rcParams['figure.dpi'] = 300
    plt.style.use(['science'])
    colors = cm.cool(np.linspace(0, 1, len(e_k_estimated)))
    fig, ax = plt.subplots()
    
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%.4f'))
    
    min_x = np.min(e_k_actual)
    max_x = np.max(e_k_actual)
    min_y = np.min(e_k_estimated)
    max_y = np.max(e_k_estimated)
    
    z = np.polyfit(e_k_actual,  e_k_estimated, 1)
    p = np.poly1d(z)
    xx = np.linspace(-p(2)/p(1), max(e_k_actual)+.0001)
    yy = np.polyval(p, xx)
    
    ax.plot(xx, yy, ls="-", color='k')
    
    for k in range(len(e_k_actual)):
        ax.scatter(e_k_actual[k], e_k_estimated[k], zorder=2, s=15, color=colors[k])

    ax.set_title(f'Actual vs. Estimated loss for k={k_:.2f}%', fontsize=8)
    ax.set_xlabel('Actual loss difference', fontsize=8)
    ax.set_ylabel('Estimated loss difference', fontsize=8)
   
    ax.set_xlim(min_x-.0001, max_x+.0001)
    ax.set_ylim(min_y-.0001, max_y+.0001)

    text = 'MAE = {:.03}\nP = {:.03}'.format(mean_absolute_error(e_k_actual, e_k_estimated), spearmanr(e_k_actual, e_k_estimated).correlation)
    print(text)
    plt.xticks(rotation = 45, fontsize=7, visible=True)
    plt.yticks(fontsize=7)

    plt.show()

#### Custom dataset

In [ ]:
 class CreateData(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.targets[idx]

        return out_data, out_label

#### Select k% of a group (based on label)

In [ ]:
def get_data(new_train_df, k):    

    selected_group = new_train_df.loc[new_train_df['label'] == 0]

    num_to_sample = round((k / 100)*len(new_train_df))

    try:
        sampled_group = selected_group.sample(n=num_to_sample, replace=False)
    except ValueError:
        sampled_group = selected_group.sample(n=num_to_sample, replace=True)
    not_selected = new_train_df.drop(sampled_group.index)

    feats = set(new_train_df.columns) - {'label'}
    selected_group_X = sampled_group[feats]
    selected_group_y = sampled_group['label']

    not_selected_group_X = not_selected[feats]
    not_selected_group_y = not_selected['label']   
    
    return selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y

### Randomized Response
Get the corresponding p and q values based on an epsilon value

In [ ]:
def get_p(epsilon):
    probability = float(E ** epsilon) / float(3 + (E ** epsilon))
    p = torch.FloatTensor([[probability, 1-probability], [1-probability, probability]])
    
    return p

### Models

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        
        self.fc1 = torch.nn.Linear(784, 4)
        self.criterion = torch.nn.CrossEntropyLoss()
        
    def forward(self, x):
        logits = self.fc1(x)

        return logits
    
    def loss(self, test_loader, print_, device):
        correct = 0
        total = 0
        all_labels = []
        all_predicted = []
        all_loss = 0
        for images, labels in test_loader:
            images = Variable(images.view(-1, 28*28)).to(device)
            labels = Variable(labels).to(device)
            outputs = self.fc1(images)
            loss = self.criterion(outputs, labels)
            all_loss += loss
            _, predicted = torch.max(outputs.data, 1)
            total+= labels.size(0)
            
            all_labels.extend(list(labels.detach().cpu().numpy()))
            all_predicted.extend(list(predicted.detach().cpu().numpy()))
            correct+= (predicted.detach().cpu().numpy() == labels.detach().cpu().numpy()).sum()
        acc = 100 * correct/total
        
        return loss/len(test_loader), acc

In [ ]:
def train(model, train_data, device):
    model.train()
    
    optimizer = torch.optim.SGD(model.parameters(), lr=.5, weight_decay=0)
    
    criterion = torch.nn.CrossEntropyLoss()

    train_dataloader = DataLoader(train_data, batch_size=100, shuffle=True)

    for itr in range(0, 10):
        itr_loss = 0
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_dataloader):
            images = Variable(images.view(-1, 28*28)).to(device)
            labels = Variable(labels).to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            itr_loss += loss
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs.data, 1)
            total+= labels.size(0)
            correct+= (predicted.detach().cpu() == labels.detach().cpu()).sum()
                        
    return model

### Influence Calculation Functions


In [ ]:
def calc_influence_single(model, epsilon, train_dataset, train_dataloader, test_dataloader, group_data, device, criterion, hessian):
    start = time.time()
    
    if hessian is None:
        s_test_vec = s_test_sample(model, test_dataloader, train_dataset, device, criterion)
    else:
        s_test_vec = hessian 
        
    grad_z_vec = grad_training([group_data[0], group_data[1]], model, device, epsilon)
    
    with torch.no_grad():
        influence = (sum([torch.sum(k * j).data for k, j in zip(grad_z_vec, s_test_vec)]) / len(train_dataset))
            
    end = time.time() - start

    return influence.cpu(), end, s_test_vec

In [ ]:
def s_test_sample(model, test_dataloader, train_dataset, device, criterion):
    scale = 25
    damp = 0.01
    recursion_depth = 7500
    r = 3
    
    inverse_hvp = [torch.zeros_like(params, dtype=torch.float) for params in model.parameters()]
    
    for i in range(r):
        hessian_loader = DataLoader(train_dataset, sampler=torch.utils.data.RandomSampler(train_dataset, True, num_samples=recursion_depth), batch_size = 1, num_workers=4)
        
        cur_estimate = s_test(test_dataloader, model, i, hessian_loader, device, damp, scale, criterion)
        
        with torch.no_grad():
            inverse_hvp = [old + (cur/scale) for old,cur in zip(inverse_hvp, cur_estimate)]
    
    with torch.no_grad():
        inverse_hvp = [component / r for component in inverse_hvp]
        
    return inverse_hvp

In [ ]:
def s_test(test_dataloader, model, i, hessian_loader, device, damp, scale, criterion):
    v = grad_z(test_dataloader, model, device, criterion)
    h_estimate = v
    
    params, names = make_functional(model)
    params = tuple(p.detach().requires_grad_() for p in params)
    
    progress_bar = tqdm(hessian_loader, desc=f"IHVP sample {i}")
    
    for i, (x_train, y_train) in enumerate(progress_bar):
        x_train = Variable(x_train.view(-1, 28*28)).to(device)
        y_train = Variable(y_train).to(device)
        
        def f(*new_params):
            load_weights(model, names, new_params)
            out = model(x_train)
            loss = criterion(out, y_train)
            return loss
    
        hv = vhp(f, params, tuple(h_estimate), strict=True)[1]
        
        with torch.no_grad():
            h_estimate = [
                _v + (1-damp) * _h_e - _hv / scale for _v, _h_e, _hv in zip(v, h_estimate, hv)
            ]
            
            if i % 100 == 0:
                norm = sum([h_.norm() for h_ in h_estimate])
                progress_bar.set_postfix({"est norm": norm.item()})
                
    with torch.no_grad():
        load_weights(model, names, params, as_params=True)
        
    return h_estimate

In [ ]:
def make_functional(model):
    orig_params = tuple(model.parameters())
    names = []
    
    for name, p in list(model.named_parameters()):
        del_attr(model, name.split("."))
        names.append(name)
    
    return orig_params, names

In [ ]:
def del_attr(obj, names):
    if len(names) == 1:
        delattr(obj, names[0])
    else:
        del_attr(getattr(obj, names[0]), names[1:])

In [ ]:
def set_attr(obj, names, val):
    if len(names) == 1:
        setattr(obj, names[0], val)
    else:
        set_attr(getattr(obj, names[0]), names[1:], val)

In [ ]:
def load_weights(model, names, params, as_params=False):
    for name, p in zip(names, params):
        if not as_params:
            set_attr(model, name.split("."), p)
        else:
            set_attr(model, name.split("."), torch.nn.Parameter(p))

In [ ]:
def grad_z(test_data, model, device, criterion):

    model.eval()

    itr_loss = 0
    for i, (images, labels) in enumerate(test_data):
        images = Variable(images.view(-1, 28*28)).to(device)
        labels = Variable(labels).to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        itr_loss += loss

    loss_ = itr_loss / len(test_data)
    return grad(loss_, model.parameters())

In [ ]:
def grad_training(train_data, model, device, epsilon):

    model.eval()

    x_train_input = torch.FloatTensor(train_data[0].values).to(device)
    y_train_input = torch.LongTensor(train_data[1].values).to(device)

    train_data = CreateData(x_train_input, y_train_input)
    train_dataloader = DataLoader(train_data, batch_size=1, shuffle=True)

    criterion = torch.nn.CrossEntropyLoss(reduction='sum')
    
    agg_loss = 0
    possible_labels = [0,1,2,3]
    for i, (image, label) in enumerate(train_dataloader):
        pert_agg_loss = 0
        
        output = model(image)
        orig_loss = criterion(output, label)

        for j in possible_labels:
            if j == label.item():
                continue
            else:
                pert_label = torch.LongTensor([j]).to(device)
               
                pert_loss = criterion(output, pert_label)
                pert_agg_loss += (pert_loss - orig_loss)
            
        itr_loss = pert_agg_loss 
        agg_loss += itr_loss
        
    loss = float(1/(3+(E ** epsilon)))*(agg_loss)
    
    to_return = grad(loss, model.parameters())
    
        
    return to_return

### Main Function

In [ ]:
def Main(dataset, epsilons, ks, num_rounds):

    device = 'cuda:4' if torch.cuda.is_available() else 'cpu'
    criterion = torch.nn.CrossEntropyLoss()
    
    all_orig_loss_e_k = []
    all_est_loss_e_k = []
    all_time = []
    
    for nr in range(num_rounds):
        print(f'\nRound {nr+1}')
        ############
        # Get data #
        ############
        print('\nGetting Data...')

        train_dataset, test_dataset = get_MNIST()

        images_train = []
        labels_train = []
        
        images_test = []
        labels_test = []
        
        for i, (image, label) in enumerate(train_dataset):
            images_train.append(image.view(-1, 28*28).tolist()[0])
            labels_train.append(label)
        
        X_train = pd.DataFrame(images_train)
        y_train = pd.DataFrame(labels_test, columns =['label'])
        
        for i, (image, label) in enumerate(test_dataset):
            images_test.append(image.view(-1, 28*28).tolist()[0])
            labels_test.append(label)
        
        X_train = pd.DataFrame(images_train)
        y_train = pd.DataFrame(labels_train, columns =['label'])
        
        X_test = pd.DataFrame(images_test)
        y_test= pd.DataFrame(labels_test, columns =['label'])
        
        x_test_input = torch.FloatTensor(X_test.values).to(device)
        y_test_input = torch.LongTensor(y_test.values).to(device)

        x_train_input = torch.FloatTensor(X_train.values).to(device)
        y_train_input = torch.LongTensor(y_train.values).to(device)
   
        new_train_df = pd.concat([X_train, y_train], axis=1)
      
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
        test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)
   
        ##############################################
        # Train original model and get original loss #
        ##############################################
        print('Training original model...')
        torch_model = LogisticRegression()
        torch.save(torch_model.state_dict(), 'initial_config_g2g.pth')
        torch_model.to(device)
        torch_model = train(torch_model, train_dataset, device)
        test_loss_ori, acc_ori = torch_model.loss(test_loader, True, device)

        e_k_act_losses = []
        e_k_est_losses = []
        influence_time = []
        
        ################################################################
        # Perform influence and retraining for all epsilons a k values #
        ################################################################
        print('\nBegining epsilon and k rounds')
        print('-----------------------------')
        for k_elem, k in enumerate(ks):
            print(f'\nk: {k}')
            hessian = None
            k_act_losses = []
            k_est_losses = []
            inf_time = []
            
            for ep_elem, ep in enumerate(epsilons):
                # Influence
                print(f'ep: {ep}')
                selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y = get_data(new_train_df, k)
                loss_diff_approx, tot_time, hessian = calc_influence_single(torch_model, ep, train_dataset, train_loader, test_loader, [selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y], device, criterion, hessian)
                loss_diff_approx = -torch.FloatTensor(loss_diff_approx).cpu().numpy()
                print(f'Approx difference: {loss_diff_approx:.5f}')
                
                # Retrain - need to actually perturb
                p_stay = ((E**ep)/(3+(E**ep)))
                p_change = (1/(3+(E**ep)))

                G_pert = []
                G_labels = selected_group_y.values.tolist()
                
                for lab in G_labels:
                    poss_labels = [0,1,2,3]
                    
                    weights = [p_change for i in range(len(poss_labels))]
                    weights[lab] = p_stay
                    
                    pert_lab = random.choices(poss_labels, weights=weights, k=1)
                    G_pert.append(pert_lab[0])
                    
                y_w_group_pert = pd.concat([not_selected_group_y, pd.DataFrame(G_pert)], axis = 0, ignore_index=True)
                y_w_group_pert = y_w_group_pert.values.tolist()
                y_w_group_pert = [yw for sublist in y_w_group_pert for yw in sublist]
               
                y_wo_pert = pd.concat([not_selected_group_y, selected_group_y], axis = 0, ignore_index=True)
                reconstructed_x = pd.concat([not_selected_group_X, selected_group_X], axis = 0, ignore_index=True)
                x_train_input_pert = torch.FloatTensor(reconstructed_x.values).to(device)
                y_train_input_pert = torch.LongTensor(y_w_group_pert).to(device)

                train_data_pert = CreateData(x_train_input_pert, y_train_input_pert)
                model_pert = LogisticRegression()
                model_pert.load_state_dict(torch.load('initial_config_g2g.pth'))
                model_pert.to(device)
                torch_model = train(model_pert, train_data_pert, device)
                test_loss_retrain, acc_retrain = model_pert.loss(test_loader, True, device)

                 # get true loss diff
                loss_diff_true = (test_loss_retrain - test_loss_ori).detach().cpu().item()
                print(f'True difference: {loss_diff_true:.5f}')
                k_act_losses.append(loss_diff_true)
                k_est_losses.append(loss_diff_approx)
                inf_time.append(tot_time)
            
            visualize_result(k_act_losses, k_est_losses, epsilons, k)
            
            e_k_act_losses.append(k_act_losses)
            e_k_est_losses.append(k_est_losses)
            influence_time.append(inf_time)
            
        all_orig_loss_e_k.append(e_k_act_losses)
        all_est_loss_e_k.append(e_k_est_losses) 
        all_time.append(influence_time)
    
    return all_orig_loss_e_k, all_est_loss_e_k, all_time

### Perform Experiment 

#### Constants

In [ ]:
epsilons = np.linspace(.001, 5, 30) #30 5
k = np.linspace(1, 30, 10) #10
rounds = 10

In [ ]:
all_orig_loss_e_k, all_est_loss_e_k, all_time = Main('mnist', epsilons, k, rounds)

with open('results/mnist/all_orig_loss_e_k_mnist.txt', "wb") as file:   #Pickling
    pickle.dump(all_orig_loss_e_k, file)

with open('results/mnist/all_est_loss_e_k_mnist.txt', "wb") as file2:   #Pickling
    pickle.dump(all_est_loss_e_k, file2)
    
with open('results/mnist/all_time_mnist.txt', "wb") as file3:   #Pickling
    pickle.dump(all_time, file3)

In [ ]:
sum_orig_loss_e_k = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]
sum_est_loss_e_k = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]
sum_time = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]

avg_orig_loss = []
avg_est_loss = []
avg_time = []

for round_ in range(len(all_orig_loss_e_k)):
    for e in range(len(epsilons)):
        for k_ in range(len(k)):
            sum_orig_loss_e_k[e][k_] = sum_orig_loss_e_k[e][k_] + all_orig_loss_e_k[round_][e][k_]
            sum_est_loss_e_k[e][k_] = sum_est_loss_e_k[e][k_] + all_est_loss_e_k[round_][e][k_]
            sum_time[e][k_] = sum_time[e][k_] + all_time[round_][e][k_]

for e in range(len(epsilons)):
    avg_orig_loss.append([ elem / len(all_orig_loss_e_k) for elem in sum_orig_loss_e_k[e]])
    avg_est_loss.append([elem/ len(all_orig_loss_e_k) for elem in sum_est_loss_e_k[e]])
    avg_time.append([elem/ len(all_orig_loss_e_k) for elem in sum_time[e]])

k_e_orig = [[] for _ in range(len(k))]
k_e_est = [[] for _ in range(len(k))]

for e in range(len(epsilons)):
    for k_ in range(len(k)):
        k_e_orig[k_].append(avg_orig_loss[e][k_])
        k_e_est[k_].append(avg_est_loss[e][k_])

averaged_time = []

for e in range(len(epsilons)):
    averaged_time.append(sum_time[e][0])

average_time_final = sum(averaged_time) / len(averaged_time)

In [ ]:
for i in range(len(k_e_orig)):
    visualize_result(k_e_orig[i], k_e_est[i], epsilons, k_[i])